In [1]:
import fastai2
from fastai2.basics import *
from fastai2.callback.all import *
from fastai2.vision.all import *

In [2]:
lbl2name = dict(
  n02086240= 'Shih-Tzu',
  n02087394= 'Rhodesian ridgeback',
  n02088364= 'Beagle',
  n02089973= 'English foxhound',
  n02093754= 'Australian terrier',
  n02096294= 'Border terrier',
  n02099601= 'Golden retriever',
  n02105641= 'Old English sheepdog',
  n02111889= 'Samoyed',
  n02115641= 'Dingo'
)

In [3]:
path = untar_data(URLs.IMAGEWOOF); path.ls()

(#3) [/home/lgvaz/.fastai/data/imagewoof/log.csv,/home/lgvaz/.fastai/data/imagewoof/val,/home/lgvaz/.fastai/data/imagewoof/train]

In [4]:
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
          get_items=get_image_files,
          splitter=GrandparentSplitter(valid_name='val'),
          get_y=[parent_label, lbl2name.get])

In [5]:
dbunch = dblock.databunch(path,
                          bs=64,
#                           item_tfms=[FlipItem(), RandomResizedCrop(128, 0.35)],
                          item_tfms=[FlipItem(), Resize(128)],
                          batch_tfms=[Normalize(*imagenet_stats)])

In [6]:
def opt_func(*args, **kwargs):
  set_trace()
  opt = RAdam(*args, mom=0.95, wd=1e-2, eps=1e-6, **kwargs)
  return Lookahead(opt)

In [7]:
class ResBlock(nn.Module):
    "Resnet block from `ni` to `nh` with `stride`"
    @delegates(ConvLayer.__init__)
    def __init__(self, expansion, ni, nh, stride=1, sa=False, sym=False,
                 norm_type=NormType.Batch, act_cls=defaults.activation, ndim=2, **kwargs):
        super().__init__()
        norm2 = NormType.BatchZero if norm_type==NormType.Batch else norm_type
        nf,ni = nh*expansion,ni*expansion
        layers  = [ConvLayer(ni, nh, 3, stride=stride, norm_type=norm_type, act_cls=act_cls, ndim=ndim, **kwargs),
                   ConvLayer(nh, nf, 3, norm_type=norm2, act_cls=None, ndim=ndim, **kwargs)
        ] if expansion == 1 else [
                   ConvLayer(ni, nh, 1, norm_type=norm_type, act_cls=act_cls, ndim=ndim, **kwargs),
                   ConvLayer(nh, nh, 3, stride=stride, norm_type=norm_type, act_cls=act_cls, ndim=ndim, **kwargs),
                   ConvLayer(nh, nf, 1, norm_type=norm2, act_cls=None, ndim=ndim, **kwargs)
        ]
        self.convs = nn.Sequential(*layers)
        self.sa = SimpleSelfAttention(nf,ks=1,sym=sym) if sa else noop
        self.idconv = noop if ni==nf else ConvLayer(ni, nf, 1, act_cls=None, ndim=ndim, **kwargs)
        self.pool = noop if stride==1 else AvgPool(2, ndim=ndim, ceil_mode=True)
        self.act = defaults.activation(inplace=True) if act_cls is defaults.activation else act_cls()

    def forward(self, x): return self.act(self.sa(self.convs(x)) + self.idconv(self.pool(x)))

In [8]:
class XResNet2(nn.Sequential):
    def __init__(self, expansion, layers, c_in=3, c_out=1000, sa=False, sym=False, act_cls=defaults.activation):
        stem = []
        sizes = [c_in,16,32,64] if c_in<3 else [c_in,32,64,64]
        for i in range(3):
            stem.append(ConvLayer(sizes[i], sizes[i+1], stride=2 if i==0 else 1, act_cls=act_cls))

        block_szs = [64//expansion,64,128,256,512] +[256]*(len(layers)-4)
        blocks = [self._make_layer(expansion, block_szs[i], block_szs[i+1], l, 1 if i==0 else 2,
                                  sa = sa if i==len(layers)-4 else False, sym=sym, act_cls=act_cls)
                  for i,l in enumerate(layers)]
        super().__init__(
            *stem,
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            *blocks,
            nn.AdaptiveAvgPool2d(1), Flatten(),
            nn.Linear(block_szs[-1]*expansion, c_out),
        )
        init_cnn(self)

    def _make_layer(self, expansion, ni, nf, blocks, stride, sa, sym, act_cls):
        return nn.Sequential(
            *[ResBlock(expansion, ni if i==0 else nf, nf, stride if i==0 else 1,
                      sa if i==blocks-1 else False, sym=sym, act_cls=act_cls)
              for i in range(blocks)])
      
fastai2.vision.models.xresnet.XResNet = XResNet2

In [9]:
def FlatCosAnnealScheduler(self:Learner, lr:float=4e-3, tot_epochs:int=1, start_pct:float=0.72,
                           curve='cosine'):
  n = len(self.dbunch.train_dl)
  anneal_start = int(n * tot_epochs * start_pct)
  batch_finish = ((n * tot_epochs) - anneal_start)
  if curve=="cosine":        curve_sched=SchedCos(lr, 0)
  elif curve=="linear":      curve_sched=SchedLin(lr, 0)
  elif curve=="exponential": curve_sched=SchedExp(lr, 0)
  else: raiseValueError(f"annealing type not supported {curve}")
  scheds = [SchedNo(lr, start_pct), curve_sched]
  scheds = {'lr': combine_scheds([start_pct, 1-start_pct], scheds)}
  return scheds

def fit_fc(learn, tot_epochs:int=1, lr:float=1e-2, start_pct:float=0.72):
    "Fit a model with Flat Cosine Annealing"
    max_lr = learn.lr
    callbacks = ParamScheduler(FlatCosAnnealScheduler(learn, lr, start_pct=start_pct, tot_epochs=tot_epochs))
    learn.fit(tot_epochs, max_lr, cbs=callbacks)

In [10]:
m = xresnet50(c_out=dbunch.c, sa=True, sym=False, act_cls=MishJit)

In [11]:
learn = Learner(dbunch, m, LabelSmoothingCrossEntropy(), metrics=[accuracy], opt_func=opt_func)

## Results

In [12]:
fit_fc(learn, 5, 4e-3)

> <ipython-input-6-1ccde1401fac>(3)opt_func()
      1 def opt_func(*args, **kwargs):
      2   set_trace()
----> 3   opt = RAdam(*args, mom=0.95, wd=1e-2, eps=1e-6, **kwargs)
      4   return Lookahead(opt)

ipdb> len(*args)
169
ipdb> kwargs
{'lr': slice(None, 0.003, None)}
ipdb> args[0]
args = ([Parameter containing:
tensor([[[[-6.2422e-01,  8.7370e-02, -3.2301e-01],
          [ 1.3750e-01, -4.5285e-01,  8.3079e-02],
          [ 1.9380e-01,  4.7274e-01,  9.5165e-02]],

         [[ 9.2870e-02,  2.0532e-02, -4.9175e-01],
          [-8.4904e-02, -3.9754e-01,  4.5248e-01],
          [-4.6566e-02,  1.2748e-01,  1.7500e-01]],

         [[-9.3041e-03, -5.2695e-01, -3.8458e-01],
          [-7.9361e-02,  7.9770e-03, -3.7394e-01],
          [-3.2569e-02,  2.2204e-01,  3.9193e-02]]],


        [[[-2.3785e-01, -3.6711e-01, -2.9071e-01],
          [-5.0316e-01,  1.9612e-01, -5.5384e-01],
          [ 4.2938e-03,  3.3820e-01, -6.5090e-01]],

         [[ 2.9304e-02,  6.6866e-02, -3.2785e-01],
       

ipdb> exit


BdbQuit: 

In [11]:
# act stem+filter size+sa, 0.7, 0.7, 0.696, 0.68, 0.7
learn.fit_flat_cos(5, 4e-3, pct_start=.72)

epoch,train_loss,valid_loss,accuracy,time
0,1.947773,1.938840,0.330000,01:13
1,1.664489,1.689755,0.474000,01:03
2,1.480462,1.474339,0.562000,01:07
3,1.383325,1.389665,0.592000,01:06
4,1.181162,1.218184,0.702000,01:02


In [11]:
# act stem + filter size, 0.67
learn.fit_flat_cos(5, 4e-3, pct_start=.72, wd=1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.956344,2.105358,0.306000,01:02
1,1.715564,2.018951,0.314000,00:55
2,1.560458,1.547535,0.516000,00:54
3,1.426260,1.507091,0.556000,00:54
4,1.230344,1.252019,0.674000,00:54


In [11]:
# act stem, 0.64, 0.66
learn.fit_flat_cos(5, 4e-3, pct_start=.72, wd=1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.931011,2.101542,0.312000,01:01
1,1.688755,2.109970,0.364000,00:56
2,1.553915,1.531896,0.540000,00:54
3,1.399058,1.523111,0.564000,00:54
4,1.224818,1.248687,0.666000,00:56


In [9]:
# default, 0.65, 0.64
learn.fit_flat_cos(5, 4e-3, pct_start=.72, wd=1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.954447,2.488014,0.268000,01:02
1,1.783182,1.990843,0.338000,00:55
2,1.613038,1.576200,0.502000,00:52
3,1.464347,1.480333,0.560000,00:58
4,1.259610,1.297685,0.648000,00:59
